In [2]:
import numpy as np

# Matricies

Recall that a 1 dimentional array in `numpy` is a vector.  It is trivial to extend what we have learnt to n-dimsional arrays.  Let's start with a simple $2 \times 2$ matrix $A$.

$ A = \begin{bmatrix}
1 & 2\\
3 & 4
\end{bmatrix}$

To create the equivalent numpy 2D array:

In [9]:
a = np.array([[1, 2], [3, 4]])
print(a)

[[1 2]
 [3 4]]


If we now inspect the `.shape` property of the array we see that the dimensions are represented in a tuple of length 2.

In [11]:
print(a.shape)

(2, 2)


Note that `numpy` defauted to `int64` as the data type.

In [12]:
print(a.dtype)

int64


To be more explicit about type we can specify it.

In [6]:
a = np.array([[1, 2], [3, 4]], dtype=np.uint8)
print(a)
print(a.dtype)

[[1 2]
 [3 4]]
uint8


## Creating multi-dimensional arrays

In essense creating a multi-dimensional array in `numpy` is no more complicated than for a vector.  Options are:
* Providing the data explicitly
* Reading a file (eg. csv)
* Built-in functions
    * `np.zeros`
    * `np.ones`
    * `np.full`
    * `np.random`
    
We can also convert a vector into a matrix using the `.reshape` method.  Here we create a vector of length 20 and rehape it into a 4 x 5 matrix.

> Note the use of the -1 in the reshape method.  Here we are telling `numpy` that to choose the correct number of rows for 5 columns of data.  Note this only works if there is no remainder!

In [20]:
as_vec = np.arange(20)
print(as_vec)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [22]:
as_matrix = as_vec.reshape(-1, 5)
print(as_matrix)
print(as_matrix.shape)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
(4, 5)


### Providing the data explicitly

This is what we have done before, converting a list of lists into an array. 
With three dimensions:

In [23]:
td = np.array([
              [[11,12], [13,14]],
              [[21,22], [23,24]],
              [[31,32], [33,34]]
              ])

print(td)

[[[11 12]
  [13 14]]

 [[21 22]
  [23 24]]

 [[31 32]
  [33 34]]]


In [ ]:
Access the data via index.  For example to access data in the first matrix, column 1 

In [24]:
td[0,1,1]

14